In [1]:
# coding:utf8
import numpy as np
import math
from math import pi
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot
from bokeh.models import LinearAxis, LogAxis, Range1d
from scipy.fftpack import fft, fftfreq, fftshift, rfft
output_file('Fourier.html')
%matplotlib notebook

In [2]:
tick = pd.read_csv('RIM6_4.trd', usecols=(0, 1, 2, 3, 4, 5, 6),
                   parse_dates={'DateTime': [0,1]}, index_col=False)
tick = tick.rename(columns={'<LAST>': 'Price', '<VOL>': 'Vol', '<DIRECTION>': 'Dir'})
tick.ix[tick.Dir == 'Sell', 'VolSell'] = tick.Vol
tick.ix[tick.Dir == 'Buy', 'VolBuy'] = tick.Vol

In [27]:
Nticks = 500     # Количество тиков для бара и ресэмплинг по тикам
Nback = 64
Ndeltas = 1000
TimeRes = '1s'  # Интервал ресэмплинга в секундах
dt = 1.0        # Interval FFT
fs = 1.0/dt      # Sampling frequency
dl = tick.fillna(0)
dl['Delta'] = dl['VolBuy'] - dl['VolSell']
dl['CumDelta'] = (dl['VolBuy'] - dl['VolSell']).cumsum()
dl['AbsCumDelta'] = abs(dl['Delta']).cumsum()
dl.iloc[:1000].to_csv('Deltas1.csv')
ff = (dl['VolSell'].iloc[0:10] != 0).sum()
ff

10

In [28]:
rows_list = []
for i in range(0, int((dl.AbsCumDelta.max()/Ndeltas))) :
    tmp = dl.loc[(dl['AbsCumDelta'] > i*Ndeltas) & (dl['AbsCumDelta'] <= i*Ndeltas + Ndeltas)]
    dict1 = {}

    op = tmp.Price.iloc[0]
    hi = tmp.Price.max()
    lo = tmp.Price.min()
    cl = tmp.Price.iloc[(len(tmp.index) - 1)]
    cumdelta = tmp.CumDelta.iloc[(len(tmp.index) - 1)]
    vBuy = tmp.VolBuy.sum()
    vSell = tmp.VolSell.sum()
    nBuy = (tmp.VolBuy.iloc[0:(len(tmp.index) - 1)] !=0).sum()
    nSell = (tmp.VolSell.iloc[0:(len(tmp.index) - 1)] !=0).sum()
    tClose = tmp.DateTime.max()
    dtime = (tmp.DateTime.max() - tmp.DateTime.min())
    dict1.update({'DateTime': tClose, 'open': op, 'high': hi,
                  'low': lo, 'close': cl, 'TickBuy': nBuy,
                  'VolBuy': vBuy, 'TickSell': nSell, 'VolSell': vSell, 'DeltaTime': dtime, 'CumDelta': cumdelta})
    rows_list.append(dict1)

res = pd.DataFrame(rows_list)

res['Dsec'] = res['DeltaTime'].dt.total_seconds()
res['TSpeedBuy'] = res['TickBuy'] / res['Dsec']
res['TSpeedSell'] = res['TickSell'] / res['Dsec']
res['VSpeedBuy'] = res['VolBuy'] / res['Dsec']
res['VSpeedSell'] = res['VolSell'] / res['Dsec']
res['OTO'] = ((res['TickBuy'] - res['TickSell']) / (res['TickBuy'] + res['TickSell']) * 0.65
              + (res['VolBuy'] - res['VolSell']) / (res['VolBuy'] + res['VolSell']) * 0.35) * 100.0
res['OTON'] = ((res['TickBuy'] * res['VolBuy'] - res['TickSell'] * res['VolSell']) /
               (res['TickBuy'] * res['VolBuy'] + res['TickSell'] * res['VolSell'])) * 100.0
res

,CumDelta,DateTime,DeltaTime,TickBuy,TickSell,VolBuy,VolSell,close,high,low,open,Dsec,TSpeedBuy,TSpeedSell,VSpeedBuy,VSpeedSell,OTO,OTON
0,-952.0,2016-04-20 10:00:01,00:00:01,8,223,24.0,976.0,90950,91860,90950,91860,1.0,8.000000,223.000000,24.000000,976.000000,-93.817835,-99.823724
1,-1501.0,2016-04-20 10:00:01,00:00:00,158,299,224.0,773.0,90860,91000,90700,90950,0.0,inf,inf,inf,inf,-39.327523,-73.441293
2,-1634.0,2016-04-20 10:00:02,00:00:01,210,303,432.0,565.0,90680,90890,90650,90860,1.0,210.000000,303.000000,432.000000,565.000000,-16.452633,-30.725617
3,-2001.0,2016-04-20 10:00:03,00:00:01,178,261,319.0,686.0,90620,90770,90620,90680,1.0,178.000000,261.000000,319.000000,686.000000,-25.070388,-51.844565
4,-2384.0,2016-04-20 10:00:05,00:00:02,162,194,309.0,692.0,90630,90650,90550,90620,2.0,81.000000,97.000000,154.500000,346.000000,-19.234305,-45.679468
5,-2337.0,2016-04-20 10:00:07,00:00:02,311,313,520.0,473.0,90640,90690,90590,90630,2.0,155.500000,156.500000,260.000000,236.500000,1.448263,4.413289
6,-2328.0,2016-04-20 10:00:10,00:00:03,267,272,508.0,499.0,90630,90700,90570,90650,3.0,89.000000,90.666667,169.333333,166.333333,-0.290158,-0.033903
7,-2267.0,2016-04-20 10:00:15,00:00:05,292,243,526.0,465.0,90680,90710,90560,90630,5.0,58.400000,48.600000,105.200000,93.000000,8.107661,15.228424
8,-2650.0,2016-04-20 10:00:22,00:00:07,171,248,311.0,694.0,90700,90750,90670,90670,7.0,24.428571,35.428571,44.428571,99.142857,-25.283416,-52.789479
9,-2696.0,2016-04-20 10:00:27,00:00:05,170,133,479.0,525.0,90850,90860,90700,90700,5.0,34.000000,26.600000,95.800000,105.000000,6.333708,7.672474


In [29]:
# res = res.set_index('DateTime')
mids = (res.open + res.close)/2
spans = abs(res.close - res.open)
inc = res.close > res.open
dec = res.open > res.close
#w = 12*60*60
w = 0.5

In [30]:
p = figure(x_range=(0, len(res.index)), plot_width=2048, plot_height=1300, title='RIM6 CandleStick')
p.y_range=Range1d(start=90500, end=96000)
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha = 0.9
p.segment(res.index.values, res.high, res.index.values, res.low, color='black')
p.rect(res.index[inc], mids[inc], w, spans[inc], fill_color='#6CF350', line_color='green')
p.rect(res.index[dec], mids[dec], w, spans[dec], fill_color='#FF0000', line_color='red')
p.extra_y_ranges = {}
p.extra_y_ranges['TickSpeed'] = Range1d(start=0, end=100)
p.extra_y_ranges['OTO'] = Range1d(start=-60, end=60)
p.extra_y_ranges['OTON'] = Range1d(start=-60, end=60)
p.extra_y_ranges['CumDelta'] = Range1d(start=-3000, end=52000)
#p.line(res.index, res.TSpeedBuy, color='blue', y_range_name='TickSpeed', alpha=0.5)
p.line(res.index, res.TSpeedBuy.rolling(window=4).median(), color='green', y_range_name='TickSpeed', alpha=0.8)
#p.line(res.index, res.TSpeedSell, color='red', y_range_name='TickSpeed', alpha=0.5)
p.line(res.index, res.TSpeedSell.rolling(window=9).median(), color='red', y_range_name='TickSpeed', alpha=0.8)
p.line(res.index, res.OTO.rolling(window=15).mean(), color='brown', y_range_name='OTO', alpha=0.9)
p.line(res.index, res.OTON.rolling(window=15).mean(), color='blue', y_range_name='OTON', alpha=0.9)
p.line(res.index, res.CumDelta, color='black', y_range_name='CumDelta', alpha=0.9)
p.add_layout(LogAxis(y_range_name='TickSpeed'), 'left')
p.add_layout(LinearAxis(y_range_name='OTO'), 'left')
p.add_layout(LinearAxis(y_range_name='OTON'), 'left')
p.add_layout(LinearAxis(y_range_name='CumDelta'), 'left')
#show(p)
#l=figure(plot_width=800, plot_height=600, title='FFT')
#l.line(resfftfreq, resfft)
#m = gridplot([[p, l]])

show(p)